In [13]:
import os
import csv

In [14]:
#random scribbling
with open("propublica_full.csv", "r", encoding='utf-8') as file:
    csvreader = csv.DictReader(file)
    for row in csvreader:
        print(row["age_cat"])
        


Greater than 45
25 - 45
Less than 25
25 - 45
25 - 45
25 - 45
25 - 45
25 - 45
Less than 25
25 - 45
25 - 45
Greater than 45
25 - 45
25 - 45
25 - 45
25 - 45
25 - 45
Greater than 45
Less than 25
25 - 45
Less than 25
Less than 25
25 - 45
25 - 45
25 - 45
25 - 45
25 - 45
25 - 45
25 - 45
Greater than 45
Greater than 45
25 - 45
Less than 25
25 - 45
25 - 45
Less than 25
Greater than 45
25 - 45
Greater than 45
25 - 45
25 - 45
25 - 45
Greater than 45
25 - 45
25 - 45
25 - 45
25 - 45
Less than 25
25 - 45
Greater than 45
Greater than 45
25 - 45
25 - 45
25 - 45
Less than 25
25 - 45
Greater than 45
25 - 45
25 - 45
Greater than 45
Greater than 45
Less than 25
25 - 45
25 - 45
25 - 45
25 - 45
Less than 25
Less than 25
25 - 45
25 - 45
25 - 45
25 - 45
Greater than 45
25 - 45
Less than 25
Less than 25
25 - 45
25 - 45
25 - 45
Less than 25
25 - 45
Less than 25
25 - 45
Greater than 45
25 - 45
Less than 25
25 - 45
25 - 45
Less than 25
25 - 45
25 - 45
25 - 45
25 - 45
Less than 25
25 - 45
25 - 45
25 - 45
25 - 45
2

Less than 25
25 - 45
25 - 45
25 - 45
25 - 45
25 - 45
25 - 45
Less than 25
Greater than 45
25 - 45
Greater than 45
Less than 25
Less than 25
25 - 45
Greater than 45
25 - 45
25 - 45
25 - 45
25 - 45
25 - 45
25 - 45
25 - 45
25 - 45
Less than 25
Less than 25
Greater than 45
Less than 25
Greater than 45
25 - 45
25 - 45
25 - 45
Greater than 45
25 - 45
25 - 45
25 - 45
Less than 25
Less than 25
25 - 45
25 - 45
Greater than 45
25 - 45
25 - 45
25 - 45
25 - 45
Less than 25
25 - 45
Less than 25
25 - 45
25 - 45
25 - 45
Greater than 45
Greater than 45
Greater than 45
Less than 25
25 - 45
Greater than 45
25 - 45
25 - 45
Less than 25
Less than 25
25 - 45
25 - 45
25 - 45
Greater than 45
Greater than 45
Greater than 45
Greater than 45
Less than 25
25 - 45
Less than 25
Less than 25
Less than 25
Greater than 45
25 - 45
25 - 45
Less than 25
25 - 45
25 - 45
Greater than 45
Less than 25
25 - 45
Less than 25
25 - 45
25 - 45
Greater than 45
Less than 25
25 - 45
Greater than 45
Less than 25
25 - 45
25 - 45
Great

25 - 45
25 - 45
Greater than 45
25 - 45
25 - 45
25 - 45
25 - 45
Greater than 45
25 - 45
25 - 45
25 - 45
25 - 45
Less than 25
Less than 25
25 - 45
25 - 45
25 - 45
25 - 45
25 - 45
25 - 45
25 - 45
25 - 45
25 - 45
Greater than 45
Greater than 45
Less than 25
25 - 45
Greater than 45
Greater than 45
25 - 45
25 - 45
25 - 45
Less than 25
25 - 45
Greater than 45
25 - 45
Less than 25
Less than 25
25 - 45
Less than 25
25 - 45
25 - 45
25 - 45
25 - 45
25 - 45
Greater than 45
Greater than 45
Less than 25
Less than 25
25 - 45
Less than 25
25 - 45
Less than 25
Less than 25
25 - 45
Greater than 45
Less than 25
25 - 45
Less than 25
25 - 45
25 - 45
25 - 45
Greater than 45
Greater than 45
25 - 45
25 - 45
Less than 25
Less than 25
Less than 25
Greater than 45
25 - 45
25 - 45
25 - 45
Less than 25
Less than 25
Less than 25
25 - 45
Less than 25
25 - 45
25 - 45
25 - 45
Greater than 45
25 - 45
25 - 45
25 - 45
25 - 45
25 - 45
25 - 45
25 - 45
25 - 45
25 - 45
25 - 45
25 - 45
25 - 45
25 - 45
Greater than 45
Less th

25 - 45
25 - 45
Less than 25
Less than 25
25 - 45
25 - 45
25 - 45
Less than 25
25 - 45
Less than 25
25 - 45
25 - 45
25 - 45
25 - 45
Less than 25
25 - 45
25 - 45
25 - 45
25 - 45
Less than 25
25 - 45
Less than 25
25 - 45
Greater than 45
Greater than 45
Less than 25
25 - 45
Less than 25
25 - 45
25 - 45
25 - 45
Greater than 45
Less than 25
Less than 25
25 - 45
25 - 45
25 - 45
25 - 45
Greater than 45
25 - 45
Less than 25
25 - 45
25 - 45
Less than 25
25 - 45
Greater than 45
25 - 45
25 - 45
25 - 45
Greater than 45
Greater than 45
25 - 45
25 - 45
Less than 25
25 - 45
Less than 25
Greater than 45
Less than 25
25 - 45
25 - 45
Greater than 45
Greater than 45
Greater than 45
25 - 45
Less than 25
25 - 45
25 - 45
25 - 45
25 - 45
25 - 45
25 - 45
Greater than 45
25 - 45
25 - 45
25 - 45
Greater than 45
Less than 25
25 - 45
25 - 45
25 - 45
25 - 45
Less than 25
Less than 25
25 - 45
25 - 45
25 - 45
25 - 45
Greater than 45
25 - 45
Greater than 45
25 - 45
Less than 25
25 - 45
Less than 25
Less than 25
25 - 

25 - 45
Less than 25
25 - 45
25 - 45
25 - 45
25 - 45
25 - 45
25 - 45
25 - 45
25 - 45
25 - 45
25 - 45
25 - 45
25 - 45
25 - 45
25 - 45
25 - 45
25 - 45
25 - 45
Greater than 45
25 - 45
25 - 45
25 - 45
25 - 45
Greater than 45
Less than 25
25 - 45
25 - 45
Less than 25
Less than 25
25 - 45
25 - 45
Greater than 45
25 - 45
25 - 45
25 - 45
Less than 25
25 - 45
25 - 45
25 - 45
25 - 45
25 - 45
25 - 45
25 - 45
25 - 45
25 - 45
25 - 45
25 - 45
Greater than 45
Greater than 45
25 - 45
25 - 45
25 - 45
Less than 25
Less than 25
Greater than 45
25 - 45
25 - 45
25 - 45
25 - 45
25 - 45
25 - 45
Less than 25
25 - 45
Greater than 45
25 - 45
25 - 45
25 - 45
25 - 45
Less than 25
Greater than 45
25 - 45
25 - 45
25 - 45
25 - 45
Less than 25
Greater than 45
25 - 45
25 - 45
25 - 45
25 - 45
25 - 45
Less than 25
25 - 45
25 - 45
Greater than 45
Less than 25
Less than 25
25 - 45
25 - 45
25 - 45
25 - 45
25 - 45
25 - 45
Less than 25
Less than 25
25 - 45
25 - 45
Greater than 45
Less than 25
Greater than 45
Less than 25
25 

25 - 45
25 - 45
Less than 25
Greater than 45
Greater than 45
25 - 45
25 - 45
25 - 45
Greater than 45
25 - 45
Less than 25
25 - 45
Greater than 45
25 - 45
25 - 45
25 - 45
Greater than 45
Less than 25
Less than 25
Greater than 45
25 - 45
25 - 45
Less than 25
25 - 45
25 - 45
25 - 45
25 - 45
25 - 45
25 - 45
25 - 45
Less than 25
25 - 45
25 - 45
Less than 25
Greater than 45
25 - 45
25 - 45
25 - 45
25 - 45
25 - 45
25 - 45
25 - 45
25 - 45
Less than 25
25 - 45
25 - 45
Greater than 45
Greater than 45
25 - 45
25 - 45
Greater than 45
25 - 45
25 - 45
Greater than 45
25 - 45
Less than 25
25 - 45
25 - 45
25 - 45
25 - 45
25 - 45
25 - 45
Less than 25
Greater than 45
25 - 45
25 - 45
Greater than 45
Less than 25
25 - 45
25 - 45
Greater than 45
25 - 45
Less than 25
Less than 25
25 - 45
25 - 45
Greater than 45
Less than 25
25 - 45
25 - 45
25 - 45
25 - 45
Less than 25
25 - 45
25 - 45
Less than 25
25 - 45
25 - 45
Greater than 45
25 - 45
Less than 25
Less than 25
25 - 45
Greater than 45
Less than 25
Greater t

In [127]:
def error(x,y,z):
    error = 0
    false_positive = 0
    false_positive_less25 = 0
    false_negative_less25 = 0
    ppv_less25 = 0
    false_positive_25_45 = 0
    false_negative_25_45 = 0
    ppv_25_45 = 0
    false_positive_great45 = 0
    false_negative_great45 = 0
    ppv_great45 = 0
    count = 6172
    with open("propublica_full.csv", "r", encoding='utf-8') as file:
        csvreader = csv.DictReader(file)
        for row in csvreader:
            if row["age_cat"] == "Less than 25":
                if int(row["decile_score"]) > x:      
                    if row["is_recid"] == "0":
                        error += 1
                        false_positive_less25 += 1
                    elif row["is_recid"] == "1":
                        ppv_less25 += 1
                elif int(row["decile_score"]) < x:
                    if row["is_recid"] == "1":
                        error += 1
                        false_negative_less25 += 1
            if row["age_cat"] == "25 - 45":
                if int(row["decile_score"]) > y:      
                    if row["is_recid"] == "0":
                        error += 1
                        false_positive_25_45 += 1
                    elif row["is_recid"] == "1":
                        ppv_25_45 += 1
                elif int(row["decile_score"]) < y:
                    if row["is_recid"] == "1":
                        error += 1
                        false_negative_25_45 += 1
            if row["age_cat"] == "Greater than 45":
                if int(row["decile_score"]) > z:      
                    if row["is_recid"] == "0":
                        error += 1
                        false_positive_great45 += 1
                    elif row["is_recid"] == "1":
                        ppv_great45 += 1
                elif int(row["decile_score"]) < z:
                    if row["is_recid"] == "1":
                        error += 1
                        false_negative_great45 += 1
    try:
        return [100*error/count, 100*abs((false_positive_less25/(false_positive_less25 + ppv_less25) - false_positive_25_45/(false_positive_25_45 + ppv_25_45)))]
    except ZeroDivisionError:
        return [0,0]
    
    
                    


In [128]:
error(6,2,3)

[31.36746597537265, 11.417135836557884]

In [129]:
error_less25 = []
error_25_45 = []
error_great25 = []
total_error = []

def errorFunc(x,y):
 for i in range(11):
     for j in range(11):
         for k in range(11):
            returned = error(i,j,k)
            x.append(returned[0])
            y.append(returned[1])

In [130]:
total_error = []
fp_less25 = []
errorFunc(total_error, fp_less25)

print(total_error)

[51.555411535968894, 42.77381723914452, 43.48671419313027, 43.30848995463383, 43.292287751134154, 43.243681140635125, 43.243681140635125, 43.69734283862606, 44.167206740116654, 44.52365521710953, 44.91250810110175, 44.37783538561244, 35.59624108878808, 36.30913804277382, 36.130913804277384, 36.1147116007777, 36.06610499027868, 36.06610499027868, 36.519766688269605, 36.98963058976021, 37.34607906675308, 37.7349319507453, 40.732339598185355, 31.950745301360985, 32.66364225534673, 32.48541801685029, 32.469215813350615, 32.42060920285159, 32.42060920285159, 32.87427090084252, 33.344134802333116, 33.70058327932599, 34.08943616331821, 40.50550874918989, 31.723914452365523, 32.43681140635126, 32.25858716785483, 32.24238496435515, 32.193778353856125, 32.193778353856125, 32.64744005184705, 33.117303953337654, 33.47375243033053, 33.86260531432275, 39.630589760207386, 30.84899546338302, 31.561892417368764, 31.383668178872327, 31.36746597537265, 31.318859364873624, 31.318859364873624, 31.772521062

In [131]:
from bokeh.plotting import figure, output_file, show
from bokeh.models import ColumnDataSource

# output to static HTML file
output_file("line.html")

data = {'x_values': total_error,
        'y_values': fp_less25}

print(fp_less25)

source = ColumnDataSource(data=data)

p = figure(plot_width=600, plot_height=600)

# add a circle renderer with a size, color, and alpha
p.circle(x='x_values', y='y_values', size=5, source=source, color="navy", alpha=0.5)

# show the results
show(p)

[9.603909867235693, 9.603909867235693, 9.603909867235693, 9.603909867235693, 9.603909867235693, 9.603909867235693, 9.603909867235693, 9.603909867235693, 9.603909867235693, 9.603909867235693, 9.603909867235693, 4.784232485844619, 4.784232485844619, 4.784232485844619, 4.784232485844619, 4.784232485844619, 4.784232485844619, 4.784232485844619, 4.784232485844619, 4.784232485844619, 4.784232485844619, 4.784232485844619, 0.06532277545279741, 0.06532277545279741, 0.06532277545279741, 0.06532277545279741, 0.06532277545279741, 0.06532277545279741, 0.06532277545279741, 0.06532277545279741, 0.06532277545279741, 0.06532277545279741, 0.06532277545279741, 3.45632369427939, 3.45632369427939, 3.45632369427939, 3.45632369427939, 3.45632369427939, 3.45632369427939, 3.45632369427939, 3.45632369427939, 3.45632369427939, 3.45632369427939, 3.45632369427939, 7.405716406829993, 7.405716406829993, 7.405716406829993, 7.405716406829993, 7.405716406829993, 7.405716406829993, 7.405716406829993, 7.405716406829993, 

In [109]:
total_error

[51.555411535968894,
 51.555411535968894,
 51.555411535968894,
 51.555411535968894,
 51.555411535968894,
 51.555411535968894,
 51.555411535968894,
 51.555411535968894,
 51.555411535968894,
 51.555411535968894,
 51.555411535968894,
 51.555411535968894,
 51.555411535968894,
 51.555411535968894,
 51.555411535968894,
 51.555411535968894,
 51.555411535968894,
 51.555411535968894,
 51.555411535968894,
 51.555411535968894,
 51.555411535968894,
 51.555411535968894,
 51.555411535968894,
 51.555411535968894,
 51.555411535968894,
 51.555411535968894,
 51.555411535968894,
 51.555411535968894,
 51.555411535968894,
 51.555411535968894,
 51.555411535968894,
 51.555411535968894,
 51.555411535968894,
 51.555411535968894,
 51.555411535968894,
 51.555411535968894,
 51.555411535968894,
 51.555411535968894,
 51.555411535968894,
 51.555411535968894,
 51.555411535968894,
 51.555411535968894,
 51.555411535968894,
 51.555411535968894,
 51.555411535968894,
 51.555411535968894,
 51.555411535968894,
 51.555411535

In [97]:
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource

data = {'x_values': [1, 2, 3, 4, 5],
        'y_values': [6, 7, 2, 3, 6]}

source = ColumnDataSource(data=data)

p = figure()
p.circle(x='x_values', y='y_values', source=source)

GlyphRenderer(id='1672', ...)